In [202]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import itertools
import math

from sklearn import base
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential

### Data wrangling

In [203]:
past = 12

In [204]:
conflict = pd.read_csv('data/conflict.csv')
pred = pd.read_csv('data/pak_df.csv')

In [205]:
conflict['event_date'] =  pd.to_datetime(conflict['event_date'], format='%d %B %Y')

In [206]:
conflict['Battles'] = conflict['event_type'] == 'Battles'
conflict['Explosions'] = conflict['event_type'] == 'Explosions/Remote violence'
conflict['Protests'] = conflict['event_type'] == 'Protests'
conflict['Riots'] = conflict['event_type'] == 'Riots'
conflict['Strategic developments'] = conflict['event_type'] == 'Strategic developments'
conflict['Violence against civilians'] = conflict['event_type'] == 'Violence against civilians'

In [207]:
conflict['month_year'] = conflict.event_date.values.astype('datetime64[M]')

In [208]:
conflict  = conflict[['month_year','latitude','longitude','Battles','Explosions','Protests','Riots',
                      'Strategic developments','Violence against civilians','fatalities','admin1','admin2','admin3']]

In [209]:
conflict['fatalities_battles'] = conflict['fatalities'] * conflict['Battles']
conflict['fatalities_explosions'] = conflict['fatalities'] * conflict['Explosions']
conflict['fatalities_protests'] = conflict['fatalities'] * conflict['Protests']
conflict['fatalities_riots'] = conflict['fatalities'] * conflict['Riots']
conflict['fatalities_strategic'] = conflict['fatalities'] * conflict['Strategic developments']
conflict['fatalities_civilian'] = conflict['fatalities'] * conflict['Violence against civilians']

In [210]:
conflict = conflict.groupby(['month_year','latitude','longitude']).agg({'Battles':'sum',
                                                             'Explosions':'sum',
                                                             'Protests':'sum',
                                                             'Riots':'sum',
                                                             'Strategic developments':'sum',
                                                             'Violence against civilians':'sum',
                                                             'fatalities_battles':'sum',
                                                             'fatalities_explosions':'sum',
                                                             'fatalities_protests':'sum',
                                                             'fatalities_riots':'sum',
                                                             'fatalities_strategic':'sum',
                                                             'fatalities_civilian':'sum'}).reset_index()

In [211]:
pred = pred.loc[:, ~pred.columns.str.contains('^Unnamed')]

In [212]:
pred = pred.rename(columns={"x": "longitude", "y": "latitude", "pak_ppp_2015": "pop", "pak07povmpi": "pov", 
                     "F182013.v4c_web.stable_lights.avg_vis": "nl", "index_1": "temp", "layer": "elev"})

In [213]:
pred = pred.dropna(subset=['pop', 'pov','nl','temp','elev']).reset_index(drop = True)

In [214]:
X = pd.merge(pred[['longitude','latitude']],conflict, on = ['longitude','latitude'], how = 'left')

In [215]:
X.loc[X.month_year.isnull(),['month_year']] = pd.to_datetime('2010-01-01')
X = X[X['month_year'] != pd.to_datetime('2019-10-01')]
X['month_year'] = X['month_year'].dt.strftime('%Y-%m')
X = X.fillna(0)

In [216]:
X = X.sort_values(by = ['longitude','latitude','month_year']).reset_index(drop=True)

In [217]:
dates = pd.date_range('2010-01-01','2019-09-01', 
              freq='MS').strftime("%Y-%m").tolist()
coor = pred[['longitude','latitude']].values.tolist()

In [218]:
a = [coor,dates]
data = list(itertools.product(*a))
df = pd.DataFrame(data, columns=['coor','month_year'])
df[['longitude','latitude']] = pd.DataFrame(df.coor.values.tolist(), index= df.index)
df = df[['longitude','latitude','month_year']].sort_values(by = ['longitude','latitude','month_year']).reset_index(drop=True)

In [219]:
X = pd.merge(df,X, on = ['longitude','latitude','month_year'], how = 'left')

In [220]:
X = X.fillna(0)

In [221]:
X = pd.merge(pred,X, on = ['longitude','latitude'], how = 'inner')

In [223]:
X['y'] = X.apply(lambda row: int((row['Battles'] + row['Explosions'] + row['Riots'] 
                                  + row['Violence against civilians']) != 0), axis=1)

In [224]:
X['month'] = X.apply(lambda row: int(row['month_year'].split('-')[1]), axis=1)
X['year'] = X.apply(lambda row: int(row['month_year'].split('-')[0]), axis=1)
X['year'] = X['year'] - 2010

In [225]:
X['y_next'] = X.shift(-1).y
X.loc[X.index.to_series().groupby([X['longitude'],X['latitude']]).last().reset_index(name='x')['x'],'y_next'] = np.nan

y = X['y_next'].values

### Transformers and estimators

In [226]:
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        features = X[self.col_names].values
        return features

In [227]:
one_hot_transformer = ColumnTransformer([("onehot", OneHotEncoder(categories='auto', sparse=False),slice(19,20))], 
                                        remainder = 'passthrough')

In [228]:
scaler = ColumnTransformer([("standard", StandardScaler(),slice(12,19))], 
                                        remainder = 'passthrough')

In [229]:
def f_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    
    beta = 1.1

    f1 = (1 + (beta**2))*p*r / ((beta**2) * p + r + K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [230]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [231]:
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

In [232]:
def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

In [278]:
class RnnEstimator(base.BaseEstimator, base.ClassifierMixin):
    
    def __init__(self, past, loss, metrics, opt, epochs):
        self.past = past
        self.loss = loss
        self.metrics = metrics
        self.opt = opt
        self.model = Sequential()
        self.epochs = epochs
    
    def fit(self, X, y):
        X_list = X.tolist()
        y_list = y.tolist()
        n = len(X_list)
        y_list = [y_list[i:i+self.past] for i in range(n - self.past + 1)]
        X_list = [X_list[i:i+self.past] for i in range(n - self.past + 1)]
        
        to_remove = [i for i in range(len(y_list)) if True in [math.isnan(item) for item in y_list[i]]]

        for index in sorted(to_remove, reverse=True):
            del y_list[index]
            del X_list[index]
        y_list = [int(item[-1]) for item in y_list]
        
        y = np.array(y_list)
        X = np.array(X_list)
        
        n_steps = X.shape[1]
        n_features = X.shape[2]
        
        self.model.add(LSTM(10, input_shape=(n_steps, n_features)))
        self.model.add(Dense(5, activation='tanh'))
        self.model.add(Dense(10, activation='tanh'))
        self.model.add(Dense(1, activation='sigmoid'))
        
        self.model.compile(optimizer=self.opt, loss=self.loss, metrics=self.metrics)
        
        self.model.fit(X, y, epochs=self.epochs)
        
        return self
    
    def score(self, X, y):
        X_list = X.tolist()
        y_list = y.tolist()
        n = len(X_list)
        y_list = [y_list[i:i+self.past] for i in range(n - self.past + 1)]
        X_list = [X_list[i:i+self.past] for i in range(n - self.past + 1)]
        
        to_remove = [i for i in range(len(y_list)) if True in [math.isnan(item) for item in y_list[i]]]

        for index in sorted(to_remove, reverse=True):
            del y_list[index]
            del X_list[index]
        y_list = [int(item[-1]) for item in y_list]
        
        y = np.array(y_list)
        X = np.array(X_list)
        
        return self.model.evaluate(X, y, batch_size = X.shape[0])
    
    def predict(self, X):
        X_list = X.tolist()
        n = len(X_list)
        X_list = [X_list[i:i+self.past] for i in range(0, n - self.past +1, self.past)]
        X = np.array(X_list)
        return self.model.predict(X)

### Test and train

In [234]:
X_train = X[X['month_year'] < '2017-07'].copy()
X_test = X[X['month_year'] >= '2017-07'].copy()

X_train['y_next'] = X_train.shift(-1).y
X_train.loc[X_train.index.to_series().groupby([X_train['longitude'], 
                                               X_train['latitude']]).last().reset_index(name='x')['x'],'y_next'] = np.nan

X_test['y_next'] = X_test.shift(-1).y
X_test.loc[X_test.index.to_series().groupby([X_test['longitude'], 
                                             X_test['latitude']]).last().reset_index(name='x')['x'],'y_next'] = np.nan

y_train = X_train['y_next'].values

y_test = X_test['y_next'].values

### Training

In [235]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
pipe = Pipeline([
        ("columns", ColumnSelectTransformer(['longitude','latitude','pop','pov','nl','temp','elev','Battles','Explosions',
             'Protests','Riots','Strategic developments','Violence against civilians',
             'fatalities_battles','fatalities_explosions','fatalities_protests','fatalities_riots',
             'fatalities_strategic','fatalities_civilian','month','year','y'])),
        ("one_hot", one_hot_transformer),
        ("scale",scaler),
        ("rnn", RnnEstimator(past, f_loss, [precision, recall, 'accuracy'], opt, 20))])

In [236]:
rnn_pipe = pipe.fit(X_train,y_train)

Epoch 1/20
8658/8658 [==============================] - 4s 468us/sample - loss: 0.3584 - precision: 0.5742 - recall: 0.7862 - acc: 0.7976
Epoch 2/20
8658/8658 [==============================] - 3s 355us/sample - loss: 0.3121 - precision: 0.6219 - recall: 0.7871 - acc: 0.8266
Epoch 3/20
8658/8658 [==============================] - 3s 332us/sample - loss: 0.3074 - precision: 0.6194 - recall: 0.8064 - acc: 0.8283
Epoch 4/20
8658/8658 [==============================] - 3s 364us/sample - loss: 0.3100 - precision: 0.6141 - recall: 0.8093 - acc: 0.8254
Epoch 5/20
8658/8658 [==============================] - 3s 353us/sample - loss: 0.2978 - precision: 0.6222 - recall: 0.8201 - acc: 0.8309
Epoch 6/20
8658/8658 [==============================] - 3s 374us/sample - loss: 0.2985 - precision: 0.6206 - recall: 0.8200 - acc: 0.8292
Epoch 7/20
8658/8658 [==============================] - 3s 389us/sample - loss: 0.2972 - precision: 0.6209 - recall: 0.8215 - acc: 0.8314
Epoch 8/20
8658/8658 [============

### Testing

In [237]:
rnn_pipe.score(X_train, y_train)

8658/8658 [==============================] - 0s 42us/sample - loss: 0.2749 - precision: 0.6368 - recall: 0.8220 - acc: 0.8392


[0.274935781955719, 0.6367891, 0.822026, 0.83922386]

In [238]:
rnn_pipe.score(X_test, y_test)

1665/1665 [==============================] - 0s 9us/sample - loss: 0.2167 - precision: 0.7849 - recall: 0.7817 - acc: 0.8691


[0.21673840284347534, 0.78486055, 0.78174603, 0.86906904]

### Train on full dataset

In [279]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
pipe = Pipeline([
        ("columns", ColumnSelectTransformer(['longitude','latitude','pop','pov','nl','temp','elev','Battles','Explosions',
             'Protests','Riots','Strategic developments','Violence against civilians',
             'fatalities_battles','fatalities_explosions','fatalities_protests','fatalities_riots',
             'fatalities_strategic','fatalities_civilian','month','year'])),
        ("one_hot", one_hot_transformer),
        ("scale",scaler),
        ("rnn", RnnEstimator(past, f_loss, [precision, recall, 'accuracy'], opt, 40))])

In [280]:
rnn_pipe = pipe.fit(X, y)

Epoch 1/40
11655/11655 [==============================] - 5s 460us/sample - loss: 0.3376 - precision: 0.6037 - recall: 0.7824 - acc: 0.8118
Epoch 2/40
11655/11655 [==============================] - 4s 385us/sample - loss: 0.3052 - precision: 0.6146 - recall: 0.8116 - acc: 0.8225
Epoch 3/40
11655/11655 [==============================] - 4s 380us/sample - loss: 0.3025 - precision: 0.6251 - recall: 0.7987 - acc: 0.8295
Epoch 4/40
11655/11655 [==============================] - 4s 381us/sample - loss: 0.2985 - precision: 0.6421 - recall: 0.7905 - acc: 0.8360
Epoch 5/40
11655/11655 [==============================] - 4s 377us/sample - loss: 0.2987 - precision: 0.6278 - recall: 0.8045 - acc: 0.8305
Epoch 6/40
11655/11655 [==============================] - 5s 401us/sample - loss: 0.2962 - precision: 0.6337 - recall: 0.8020 - acc: 0.8341
Epoch 7/40
11655/11655 [==============================] - 4s 377us/sample - loss: 0.2962 - precision: 0.6369 - recall: 0.8008 - acc: 0.8360
Epoch 8/40
11655/116

### Compare with naive model

In [241]:
rnn_pipe.score(X, y)

11655/11655 [==============================] - 0s 34us/sample - loss: 0.2778 - precision: 0.6413 - recall: 0.8087 - acc: 0.8378


[0.27782291173934937, 0.64129555, 0.80871344, 0.83775204]

In [242]:
X_naive = X.copy()

X_naive = X_naive.dropna(subset=['y_next']).reset_index(drop = True)

true_positives = sum((X_naive['y_next'] * X_naive['y']).values)
predicted_positives = sum(X_naive['y'].values)
possible_positives = sum((X_naive['y_next']).values)

In [243]:
true_positives / predicted_positives #precision

0.6294670846394984

In [244]:
true_positives / possible_positives #recall

0.6275

### Predictions for September 2019

In [314]:
X_pred = X.copy()
X_pred = X_pred[(X_pred['month_year'] >= '2018-09') & (X_pred['month_year'] < '2019-09')].reset_index(drop=True)

predictions = rnn_pipe.predict(X_pred)
lat_lon = X_pred.iloc[::12, :].reset_index(drop=True)[['longitude','latitude']]
lat_lon['pred'] = predictions

### Naive Predictions for September 2018

In [316]:
naive = conflict[conflict['month_year'] == pd.to_datetime('2019-08-01')]
naive = pd.merge(pred[['longitude','latitude']],naive, on = ['longitude','latitude'], how = 'left')
naive.loc[naive.month_year.isnull(),['month_year']] = pd.to_datetime('2019-08-01')
naive['month_year'] = naive['month_year'].dt.strftime('%Y-%m')
naive = naive.fillna(0)
naive = naive.sort_values(by = ['longitude','latitude','month_year']).reset_index(drop=True)
naive['naive_pred'] = naive.apply(lambda row: int((row['Battles'] + row['Explosions'] + row['Riots'] 
                                                   + row['Violence against civilians']) != 0), axis=1)

naive = naive[['longitude','latitude','naive_pred']]

### Actual values for September 2019

In [318]:
actual = conflict[conflict['month_year'] == pd.to_datetime('2019-09-01')]
actual = pd.merge(pred[['longitude','latitude']],actual, on = ['longitude','latitude'], how = 'left')
actual.loc[actual.month_year.isnull(),['month_year']] = pd.to_datetime('2019-09-01')
actual['month_year'] = actual['month_year'].dt.strftime('%Y-%m')
actual = actual.fillna(0)
actual = actual.sort_values(by = ['longitude','latitude','month_year']).reset_index(drop=True)
actual['actual'] = actual.apply(lambda row: int((row['Battles'] + row['Explosions'] + row['Riots'] 
                                                 + row['Violence against civilians']) != 0), axis=1)

actual = actual[['longitude','latitude','actual']]

In [319]:
compare = pd.merge(lat_lon, naive, on = ['longitude','latitude'])
compare = pd.merge(compare, actual, on = ['longitude','latitude'])

true_positives = sum((compare['actual'] * (compare['pred'] > 0.5).astype('int')).values)
predicted_positives = sum(((compare['pred'] > 0.5).astype('int')).values)
possible_positives = sum((compare['actual']).values)

##### Precision and Recall for RNN model

In [320]:
true_positives / predicted_positives #precision

0.8823529411764706

In [321]:
true_positives / possible_positives #recall

0.9090909090909091

##### Precision and Recall for Naive model

In [322]:
true_positives = sum((compare['actual'] * compare['naive_pred']).values)
predicted_positives = sum((compare['naive_pred']).values)
possible_positives = sum((compare['actual']).values)

In [323]:
true_positives / predicted_positives #precision

0.8125

In [324]:
true_positives / possible_positives #recall

0.7878787878787878